In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
import os
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers
# import keras_tuner as kt
import seaborn as sns
from datetime import datetime
import optuna

In [4]:
os.chdir('drive/My Drive/Nbeats')

In [5]:
ipFiles = os.listdir('dataset50')
len(ipFiles)

50

In [6]:
with open('ipfiles.pickle','wb') as f:
  pickle.dump(ipFiles,f)

In [7]:
def oobsplit(pddata:pd.DataFrame):
  ''' splitting 75% for training and 25% for  testing'''
  splitSize = int(len(pddata)*0.75)
  data = pddata.iloc[:splitSize,:]
  oofdata = pddata.iloc[splitSize:,:]

  return data,oofdata


In [8]:
def make_window(data:pd.DataFrame):
  ''' creating columns with previous timestamp's values'''
  t2 = data.copy()
  t2 = t2[['value']]
  # Add windowed columns
  for i in range(50): # Shifting values for each step to create 50 subsequent cols.
    t2[f"value-{i+1}"] = t2["value"].shift(periods=i+1)
  return t2.dropna()

In [9]:
def WindowHorizon(data,WINDOW_SIZE,HORIZON):
  '''first breaking the training data into Train and
  validation data using split_size. Then using the window+horizon number of 
  columns from the 50 columns we have generated in make_window func'''
  split_size = int(len(data)*0.8)
  
  data = data.iloc[:,:WINDOW_SIZE+HORIZON]

  xtr = data.iloc[:split_size,-WINDOW_SIZE:]
  ytr = data.iloc[:split_size,:HORIZON]
  xts = data.iloc[split_size:,-WINDOW_SIZE:]
  yts = data.iloc[split_size:,:HORIZON]

  return xtr,ytr,xts,yts

  

In [10]:
def tensorize_data(xtr:pd.DataFrame,ytr:pd.DataFrame,xts:pd.DataFrame,yts:pd.DataFrame,BATCH_SIZE):
  '''generating the tensor out of data to feed to neural network'''
  
  train_features_dataset = tf.data.Dataset.from_tensor_slices(xtr)
  train_labels_dataset = tf.data.Dataset.from_tensor_slices(ytr)

  test_features_dataset = tf.data.Dataset.from_tensor_slices(xts)
  test_labels_dataset = tf.data.Dataset.from_tensor_slices(yts)

  train_dataset = tf.data.Dataset.zip((train_features_dataset, train_labels_dataset))
  test_dataset = tf.data.Dataset.zip((test_features_dataset, test_labels_dataset))

  train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
  test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

  return train_dataset, test_dataset

In [11]:
def prep_data(data,WINDOW_SIZE,HORIZON,BATCH_SIZE):
  ''' calling all the 3 functions to prepare the data at
  one place'''
  data = make_window(data)
  xtr,ytr,xts,yts = WindowHorizon(data,WINDOW_SIZE,HORIZON)
  train_dataset, test_dataset = tensorize_data(xtr,ytr,xts,yts,BATCH_SIZE)

  return train_dataset, test_dataset




In [12]:
class NBeatsBlock(tf.keras.layers.Layer):
  def __init__(self, 
               input_size: int,
               theta_size: int,
               horizon: int,
               n_neurons: int,
               n_layers: int,
               **kwargs):
    super().__init__(**kwargs)
    self.input_size = input_size
    self.theta_size = theta_size
    self.horizon = horizon
    self.n_neurons = n_neurons
    self.n_layers = n_layers

    # Block contains stack of 4 fully connected layers each has ReLU activation
    self.dense_layers = [tf.keras.layers.Dense(n_neurons, activation="relu") for _ in range(n_layers)]
    # Output of block is a theta layer with linear activation
    self.theta_layer = tf.keras.layers.Dense(theta_size, activation="linear", name="theta")

  def call(self, inputs): # the call method is what runs when the layer is called 
    x = inputs 
    for layer in self.dense_layers: 
      x = layer(x)
    theta = self.theta_layer(x) 
    # Output the backcast and forecast from theta
    backcast, forecast = theta[:, :self.input_size], theta[:, -self.horizon:]
    return backcast, forecast

In [13]:
def make_preds(model, input_data):
   return tf.squeeze(model.predict(input_data))

In [14]:
def evaluate_preds(y_true, y_pred):
  # float32 for metric calculations
  y_true = tf.cast(y_true, dtype=tf.float32)
  y_pred = tf.cast(y_pred, dtype=tf.float32)

  # Calculate all the req metrics
  # mae = tf.keras.metrics.mean_absolute_error(y_true, y_pred)
  # mse = tf.keras.metrics.mean_squared_error(y_true, y_pred) # puts and emphasis on outliers (all errors get squared)
  # rmse = tf.sqrt(mse)
  mape = tf.keras.metrics.mean_absolute_percentage_error(y_true, y_pred).numpy().mean()
  
  return mape
  
  # return {"mae": mae.numpy(),
  #         "mse": mse.numpy(),
  #         "rmse": rmse.numpy(),
  #         "mape": mape.numpy(),
  #         }

In [15]:
def obj_func(trial,data):
  ''' defining the objective func for optuna study to minimize 
  the loss returned'''

  
  WINDOW_SIZE= trial.suggest_int('WINDOW_SIZE',5,38,3)
  HORIZON = trial.suggest_int('HORIZON',1,5,1)

  N_EPOCHS = 1000
  N_NEURONS = trial.suggest_categorical('N_NEURONS',[128,256,512,1024]) 
  N_LAYERS = 4
  N_STACKS = trial.suggest_categorical('N_STACKS',[10,20,30,40])
  BATCH_SIZE = 128 

  INPUT_SIZE = WINDOW_SIZE 
  THETA_SIZE = INPUT_SIZE + HORIZON
  INPUT_SIZE, THETA_SIZE


  tf.random.set_seed(5)

  data,_ = oobsplit(data)
  train_dataset,test_dataset = prep_data(data,WINDOW_SIZE,HORIZON,BATCH_SIZE)

  
  
  #Nbeats architecture
  nbeats_block_layer = NBeatsBlock(input_size=INPUT_SIZE,
                                  theta_size=THETA_SIZE,
                                  horizon=HORIZON,
                                  n_neurons=N_NEURONS,
                                  n_layers=N_LAYERS,
                                  name="InitialBlock")

  stack_input = layers.Input(shape=(INPUT_SIZE), name="stack_input")

  backcast, forecast = nbeats_block_layer(stack_input)
  residuals = layers.subtract([stack_input, backcast], name=f"subtract_00") 

  for i, _ in enumerate(range(N_STACKS-1)): # first stack is already creted in (3)

    backcast, block_forecast = NBeatsBlock(
        input_size=INPUT_SIZE,
        theta_size=THETA_SIZE,
        horizon=HORIZON,
        n_neurons=N_NEURONS,
        n_layers=N_LAYERS,
        name=f"NBeatsBlock_{i}"
    )(residuals) # pass it in residuals (the backcast)

    residuals = layers.subtract([residuals, backcast], name=f"subtract_{i}") 
    forecast = layers.add([forecast, block_forecast], name=f"add_{i}")

  model = tf.keras.Model(inputs=stack_input, 
                          outputs=forecast, 
                          name="model_N-BEATS")

  model.compile(loss="mape",
                  optimizer=tf.keras.optimizers.Adam(0.001),
                  )

  model.fit(train_dataset,
              epochs=N_EPOCHS,
              validation_data=test_dataset,
              verbose=0, 
              # callbacks=[create_model_checkpoint(model_name=stack_model.name)] # saving model every epoch takes too much time
              callbacks=[tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=True),
                        tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=10, verbose=0),
                        #  tf.keras.callbacks.ModelCheckpoint(FILEPATH, monitor='val_loss',verbose=0,save_best_only=True, mode='min') # need to write a get_config method for this
                        ])
  

  preds = make_preds(model, test_dataset)
  _,_,_,ytrue = WindowHorizon(make_window(data),WINDOW_SIZE,HORIZON)

    
  return evaluate_preds(ytrue,preds)
   


In [16]:
def hypertune(data,i,model_arr):
  '''hypertunning the window size, horizon size, neurons and
  number of stack to use for each dataset'''

  study_name = f'model_{i}'
  study = optuna.create_study(study_name=study_name,storage=f'sqlite:///modeldb/model_{i}.db',
                              pruner=optuna.pruners.MedianPruner(n_warmup_steps=10),load_if_exists=True, direction="minimize")


  study.optimize(lambda trial : obj_func(trial,data),gc_after_trial=True,timeout=1200, n_trials= 10)
  print(study.best_trial.params)
  model_arr.append(study.best_trial.params)#an array/list to store the best values model wise
  with open('model_para_arr.pickle','wb') as f:
    pickle.dump(model_arr,f)

In [17]:
model_arr = []
for i,files in enumerate(ipFiles):
  pddata = pd.read_csv(f'dataset50/{files}',index_col = 0)
  hypertune(pddata,i,model_arr)
  

[I 2022-05-03 15:30:56,076] A new study created in RDB with name: model_0
[I 2022-05-03 15:31:35,855] Trial 0 finished with value: 18034420.0 and parameters: {'WINDOW_SIZE': 20, 'HORIZON': 5, 'N_NEURONS': 512, 'N_STACKS': 40}. Best is trial 0 with value: 18034420.0.
[I 2022-05-03 15:32:31,422] Trial 1 finished with value: 11455383.0 and parameters: {'WINDOW_SIZE': 14, 'HORIZON': 2, 'N_NEURONS': 256, 'N_STACKS': 40}. Best is trial 1 with value: 11455383.0.
[I 2022-05-03 15:32:46,124] Trial 2 finished with value: 31890462.0 and parameters: {'WINDOW_SIZE': 20, 'HORIZON': 2, 'N_NEURONS': 128, 'N_STACKS': 20}. Best is trial 1 with value: 11455383.0.
[I 2022-05-03 15:33:33,020] Trial 3 finished with value: 23291974.0 and parameters: {'WINDOW_SIZE': 26, 'HORIZON': 2, 'N_NEURONS': 1024, 'N_STACKS': 40}. Best is trial 1 with value: 11455383.0.


[I 2022-05-03 15:33:48,810] Trial 4 finished with value: 577721.5 and parameters: {'WINDOW_SIZE': 5, 'HORIZON': 3, 'N_NEURONS': 256, 'N_STACKS': 10}. Best is trial 4 with value: 577721.5.


[I 2022-05-03 15:34:08,019] Trial 5 finished with value: 67545352.0 and parameters: {'WINDOW_SIZE': 26, 'HORIZON': 1, 'N_NEURONS': 128, 'N_STACKS': 30}. Best is trial 4 with value: 577721.5.
[I 2022-05-03 15:34:35,489] Trial 6 finished with value: 54345300.0 and parameters: {'WINDOW_SIZE': 23, 'HORIZON': 2, 'N_NEURONS': 128, 'N_STACKS': 40}. Best is trial 4 with value: 577721.5.
[I 2022-05-03 15:35:23,281] Trial 7 finished with value: 25552072.0 and parameters: {'WINDOW_SIZE': 35, 'HORIZON': 1, 'N_NEURONS': 1024, 'N_STACKS': 40}. Best is trial 4 with value: 577721.5.
[I 2022-05-03 15:35:30,448] Trial 8 finished with value: 16838068.0 and parameters: {'WINDOW_SIZE': 32, 'HORIZON': 1, 'N_NEURONS': 256, 'N_STACKS': 10}. Best is trial 4 with value: 577721.5.
[I 2022-05-03 15:36:29,176] Trial 9 finished with value: 748348.125 and parameters: {'WINDOW_SIZE': 5, 'HORIZON': 2, 'N_NEURONS': 512, 'N_STACKS': 40}. Best is trial 4 with value: 577721.5.


{'HORIZON': 3, 'N_NEURONS': 256, 'N_STACKS': 10, 'WINDOW_SIZE': 5}


[I 2022-05-03 15:36:30,516] A new study created in RDB with name: model_1
[I 2022-05-03 15:36:41,874] Trial 0 finished with value: 0.6557193398475647 and parameters: {'WINDOW_SIZE': 17, 'HORIZON': 4, 'N_NEURONS': 256, 'N_STACKS': 10}. Best is trial 0 with value: 0.6557193398475647.
[I 2022-05-03 15:36:57,214] Trial 1 finished with value: 2.731217861175537 and parameters: {'WINDOW_SIZE': 35, 'HORIZON': 1, 'N_NEURONS': 128, 'N_STACKS': 20}. Best is trial 0 with value: 0.6557193398475647.
[I 2022-05-03 15:37:53,625] Trial 2 finished with value: 0.6274402737617493 and parameters: {'WINDOW_SIZE': 20, 'HORIZON': 4, 'N_NEURONS': 512, 'N_STACKS': 40}. Best is trial 2 with value: 0.6274402737617493.
[I 2022-05-03 15:38:16,128] Trial 3 finished with value: 1.2038347721099854 and parameters: {'WINDOW_SIZE': 5, 'HORIZON': 3, 'N_NEURONS': 256, 'N_STACKS': 30}. Best is trial 2 with value: 0.6274402737617493.
[I 2022-05-03 15:38:50,281] Trial 4 finished with value: 1.1561100482940674 and parameters: 

...

{'HORIZON': 4, 'N_NEURONS': 1024, 'N_STACKS': 20, 'WINDOW_SIZE': 17}


[I 2022-05-03 19:24:58,991] A new study created in RDB with name: model_48
[I 2022-05-03 19:25:05,946] Trial 0 finished with value: 10.76737117767334 and parameters: {'WINDOW_SIZE': 8, 'HORIZON': 4, 'N_NEURONS': 256, 'N_STACKS': 10}. Best is trial 0 with value: 10.76737117767334.
[I 2022-05-03 19:25:18,689] Trial 1 finished with value: 8.798286437988281 and parameters: {'WINDOW_SIZE': 26, 'HORIZON': 5, 'N_NEURONS': 256, 'N_STACKS': 20}. Best is trial 1 with value: 8.798286437988281.
[I 2022-05-03 19:26:03,506] Trial 2 finished with value: 7.3372063636779785 and parameters: {'WINDOW_SIZE': 26, 'HORIZON': 4, 'N_NEURONS': 1024, 'N_STACKS': 40}. Best is trial 2 with value: 7.3372063636779785.
[I 2022-05-03 19:26:23,010] Trial 3 finished with value: 7.70209264755249 and parameters: {'WINDOW_SIZE': 29, 'HORIZON': 3, 'N_NEURONS': 512, 'N_STACKS': 30}. Best is trial 2 with value: 7.3372063636779785.
[I 2022-05-03 19:26:45,428] Trial 4 finished with value: 8.243721008300781 and parameters: {'WI

{'HORIZON': 2, 'N_NEURONS': 512, 'N_STACKS': 40, 'WINDOW_SIZE': 26}


[I 2022-05-03 19:28:36,907] A new study created in RDB with name: model_49
[I 2022-05-03 19:29:05,778] Trial 0 finished with value: 135920.8125 and parameters: {'WINDOW_SIZE': 20, 'HORIZON': 4, 'N_NEURONS': 128, 'N_STACKS': 10}. Best is trial 0 with value: 135920.8125.
[I 2022-05-03 19:29:34,629] Trial 1 finished with value: 114934.5234375 and parameters: {'WINDOW_SIZE': 8, 'HORIZON': 1, 'N_NEURONS': 128, 'N_STACKS': 30}. Best is trial 1 with value: 114934.5234375.
[I 2022-05-03 19:29:51,422] Trial 2 finished with value: 29216.60546875 and parameters: {'WINDOW_SIZE': 11, 'HORIZON': 5, 'N_NEURONS': 512, 'N_STACKS': 10}. Best is trial 2 with value: 29216.60546875.
[I 2022-05-03 19:30:05,141] Trial 3 finished with value: 5157.17236328125 and parameters: {'WINDOW_SIZE': 5, 'HORIZON': 4, 'N_NEURONS': 256, 'N_STACKS': 10}. Best is trial 3 with value: 5157.17236328125.
[I 2022-05-03 19:30:18,772] Trial 4 finished with value: 39644.44140625 and parameters: {'WINDOW_SIZE': 8, 'HORIZON': 4, 'N_N

{'HORIZON': 4, 'N_NEURONS': 512, 'N_STACKS': 20, 'WINDOW_SIZE': 17}
